# PHÂN TÍCH KHÁM PHÁ DỮ LIỆU

### 1. Nhập các thư viện cần thiết

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

# Đảm bảo rằng các hình ảnh hiển thị rõ ràng
%matplotlib inline
sns.set(style="whitegrid")

### 2. Đọc dữ liệu

In [ ]:
# Đường dẫn tới dữ liệu thô
data_path = "../data/raw/creditcard.csv"
figure_dir = "../reports/figures"
df = pd.read_csv(data_path)


### 3. Tổng quan dữ liệu

In [ ]:
print("Kích thước của dataset:", df.shape)
print("\nThông tin chi tiết về dataset:")
df.info()


print("\nXem qua 5 dòng đầu tiên của dataset:")
df.head()

### Highlights

1. **Kích thước dữ liệu**:  
   - Tập dữ liệu chứa **284,807 giao dịch** với **31 cột đặc trưng**.  

2. **Kiểu dữ liệu**:  
   - Tất cả các cột là kiểu `float64`, ngoại trừ **Class** (kiểu số nguyên).  

3. **Đặc trưng ẩn danh**:  
   - 28 cột (`V1`, `V2`, ..., `V28`) là các đặc trưng đã được giảm chiều bằng PCA.  
   - Các giá trị trong những cột này không mang ý nghĩa logic trực tiếp.

4. **Các cột quan trọng**:  
   - **TIME**, **AMOUNT**, và **CLASS** là những cột có thể phân tích sâu.

---


### 4. Thống kê mô tả

In [ ]:
print("\nThống kê mô tả các đặc trưng liên tục:")
print(df[['Time','Amount']].describe())

#### Highlights
Thời gian (Time): Thời gian giao dịch dao động từ 0 đến ~48 giờ, với thời điểm trung bình ~26 giờ.  
Số tiền (Amount): 75% các giao dịch dưới 77.17 đơn vị tiền tệ, nhưng có giao dịch cao nhất lên đến 25,691.16.
Chỉ ra phần lớn giao dịch có giá trị nhỏ, trong khi các giao dịch có giá trị lớn có thể cần chú ý đặc biệt vì khả năng liên quan đến gian lận cao hơn.

In [ ]:
# 4.1 Kiểm tra giá trị thiếu
print("\nKiểm tra các giá trị thiếu trong tập dữ liệu:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Dữ liệu không có giá trị thiếu, đảm bảo tính toàn vẹn của phân tích.

In [ ]:
# 4.2 Tóm tắt thông tin chính
print("\nSố lượng giá trị duy nhất trong từng cột:")
print(df.nunique())

### 5. Phân phối dữ liệu

#### 5.1. Phân tích nhãn (class)

In [ ]:
# Tần suất của các nhãn (Class)
class_counts = df['Class'].value_counts()
print("\nPhân phối nhãn (Class):")
print(class_counts)

# Vẽ biểu đồ hình tròn thể hiện tỷ lệ các nhãn
labels = ['Không gian lận (Class 0)', 'Gian lận (Class 1)']
sizes = [class_counts[0], class_counts[1]]
colors = ['#66b3ff', '#ff9999']

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, explode=(0, 0.1))
plt.title("Tỷ lệ giao dịch gian lận và không gian lận", fontsize=16)
plt.axis('equal')  # Đảm bảo hình tròn hoàn chỉnh
plt.savefig(os.path.join(figure_dir, "1_pie_class_ling.png"))
plt.show()

#### Highlights 
Bộ dữ liệu này có 492 vụ gian lận trong số 284315 giao dịch.  
Tập dữ liệu có tính mất cân bằng cao, tỷ lệ gian lận chiếm xấp xỉ 0,2% tổng số giao dịch. Hầu hết các giao dịch đều không gian lận.

#### 5.2. Phân phối thời gian giao dịch (Time)

In [ ]:
plt.figure(figsize=(10, 6))
sns.distplot(df['Time'], bins=50)
plt.title("Phân phối của thời gian giao dịch (Time)", fontsize=16)
plt.xlabel("Thời gian (giây)", fontsize=12)
plt.ylabel("Số lượng giao dịch", fontsize=12)
plt.savefig(os.path.join(figure_dir, "2_distribution_time.png"))
plt.show()

#### Highlights

Biểu đồ cho thấy có hai đỉnh trong biểu đồ và có một số đỉnh cục bộ.  
Chúng ta có thể coi đây là thời gian trong ngày, cao điểm là thời điểm ban ngày và độ sâu là thời gian ban đêm.

### 5.3. Phân phối số tiền giao dịch (Amount)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['Amount'], bins=50, kde=True, color="green")
plt.title("Phân phối của số tiền giao dịch (Amount)", fontsize=16)
plt.xlabel("Amount", fontsize=12)
plt.ylabel("Tần suất", fontsize=12)
plt.savefig(os.path.join(figure_dir, "3_distribution_amount.png"))
plt.show()

#### Highlights  
Hầu hết số tiền giao dịch nằm trong khoảng từ 0 đến khoảng 3000 và có một số ngoại lệ cho các giao dịch số lượng thực sự lớn.


### 6. Phân tích ngoại lai (Outliers)

In [ ]:
# Vẽ boxplot để trực quan hóa ngoại lai trên Amount
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['Amount'], color="orange")
plt.title("Boxplot của số tiền giao dịch (Amount)", fontsize=16)
plt.xlabel("Amount", fontsize=12)
plt.savefig(os.path.join(figure_dir, "4_boxplot_amount.png"))
plt.show()

#### Highlights
Có một số ngoại lệ cho các giao dịch số lượng thực sự lớn và việc loại bỏ các ngoại lệ đó trong phân tích có thể gây ảnh hưởng đến mô hình sau này. Vậy nên, chúng ta phải kiểm tra những ngoại lệ này không phải là giao dịch gian lận hay không. Nói chung, các giao dịch gian lận với số tiền rất lớn và việc xóa chúng khỏi dữ liệu có thể khiến mô hình dự đoán trở nên sai lầm.

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='Class', y='Amount',data = df)
plt.title('Phân phối số tiền cho các giao dịch gian lận và không giân lận')
plt.savefig(os.path.join(figure_dir, "5_boxplot_class.png"))
plt.show()

#### Highlight
Theo biểu đồ trên, về cơ bản có thể thấy giao dịch gian lận không bị ảnh hưởng bởi các giá trị ngoại lệ, vậy nên xây dựng một mô hình dự đoán giao dịch là gian lận mà không bị ảnh hưởng. Có thể không tốt khi mô hình được huấn luyện dựa trên những giá trị ngoại lệ này.

### 7. Phân loại biến

In [ ]:
categorical_features = ['Class']
continuous_features = [col for col in df.columns if col not in categorical_features and col != 'Time']

print("\nCác đặc trưng phân loại:", categorical_features)
print("Các đặc trưng liên tục:", continuous_features)

### 8. Phân tích tương quan

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=False, fmt=".2f")
plt.title("Ma trận tương quan giữa các đặc trưng", fontsize=16)
plt.savefig(os.path.join(figure_dir, "6_correlation_matrix.png"))
plt.show()

#### Highlights 
Ma trận tương quan cho thấy các mối quan hệ giữa các đặc trưng.  
Amount và Class có tương quan rất nhỏ. V2, V4, V11 cũng có mối tương quan với Class.

### 9. Phân phối các đặc trưng ẩn danh (V1 - V28)

In [ ]:
plt.figure(figsize=(15, 12))
for i, feature in enumerate(df.columns[1:29], 1):  # Chọn cột từ V1 đến V28
    plt.subplot(6, 5, i)
    sns.histplot(df[feature], bins=30, kde=True, color="blue")
    plt.title(feature)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "7_correlation_matrix.png"))
plt.show()
